# RNAseq lesson

Welcome to this RNA-seq lesson, we will give a basic rundown of procces of going form your raw data to differential gene expression. For this we will use a couple of different tools:<br> 
<br>
On a <b>Mac</b> or <b>linux</b> computer we just need <a href="https://www.rstudio.com/products/rstudio/download/">Rstudio</a> and we need
  <a href="https://www.anaconda.com/download/#macos">anaconda</a>
from which we will get the other packages.
. <br>
While on a <b>windows</b> computer you will need to get 
 <a href="http://hannonlab.cshl.edu/fastx_toolkit/download.html">Fastx_toolkit</a>,
 <a href="https://pachterlab.github.io/kallisto/download">kallisto</a>
and 
 <a href="https://pachterlab.github.io/sleuth/download">sleuth</a>
from their own websites.<br>

<b>Anaconda</b> - This will allow us to get the other packages easy without hickups. be sure to add the "bioconda" channel. After which packages can be found by going to "environments" then search package your package of choice. from here install "fastx_toolkit", "kallisto" and "sleuth"<br>
<b>FASTQC</b>  - to check the quality of the reads. This can be gotten from anaconda. We will not go in depth how to work with this package here, as a previous lesson already covers the subject: <a href="http://www.datacarpentry.org/wrangling-genomics/00-quality-control/">here</a><br>
<b>FASTX-toolkit</b> - A toolkit to filter your .fasta files in multiple ways. An alternative is trimmomatic which has many of the same features. Can be gotten from anaconda.<br>
<b>Kallisto</b> - This is the alligner we will be using during this example. Can be gotten from anaconda. <br>
<b>Sleuth</b> - the accompanying software for Kallisto, this can be run from R(studio) or the shell, we will use Rstudio in the example, if your don't have Rstudio already it can also be gotten from anaconda. How to install it will be covered when we get there.<br>
<br>
For each step we have providede example data, that can be downloaded from here 
<a href="https://www.dropbox.com/sh/lrl7q4c6yv4y3va/AABsQTqVj47a5C_o8MxL-n2xa?dl=0">here</a>. If you save all the data to a folder, and set this as your "working directory" all command should be able to run.





The data we have provided are Arabidopsis samples from Max his experiment. In which he is comparing different PLC mutants their roots and shoots, under control and drought conditions.<br> In order to keep this lesson concise we have only provided you with 1% of the data (100.000 reads/sample) of Wt shoot samples under control and drought conditions.

### FastQC
We will not delve into the possibilities of FastQC here, for more information you can check out <a href="http://www.datacarpentry.org/wrangling-genomics/00-quality-control/">This</a> lesson.<br>
In anaconda you can load the FASTQC package or download it <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/">here</a><br>
The raw data files have some reads that have a low quality score, and some reads that are to small to allign properly to the transcripts, so the next step will be to remove these reads.<br>


We want to do 2 things with our sample, we want to remove all reads that are smaller than 25 nucleotides and have a quality score of 17 or less. We choose 17 in this case since we are working with data sequenced using ion-torrent sequencing. If your own data set has used some other sequencing platform you might want to use a different cut off value (illumina is around 30, but very much depended on your samples.)<br>
We will use the fastx-toolkit to do this. Make sure you have it installed first. And make sure you are working in the right folder (this should be the folder where the .ipynb file is located)

In [ ]:
%%bash
ls

We can get rid of reads smaller than 25 nucleotides with the following command. 

In [ ]:
%%bash
ls
fastx_clipper -l 25 -i fastq-files/S01*.fastq -o filtered/S01*.fastq

In the same manner we can filter out the reads with low quality scores. In this command the q stands for the quality that we want and the p stands for the percentage of the reads that has to be this quality to be cut out

In [66]:
%%bash
fastq_quality_filter -q 17 -p 75 -i fastq-files/S01*.fastq -o filtered/S01*.fastq
#note that this will overwrite our previous filtered file

Now that we know both commands we can put them together and loop them so we do the filtering for all .fasta files

In [72]:
%%bash
cd fastq-files
for filename in S*.fastq; 
    do fastx_clipper -l 25 -i $filename | fastq_quality_filter -q 17 -p 75 -o ../filtered/$filename-filtered.fastq; 
done

# Kallisto

Now that we have filtered the data we can use it to allign to a reference genome. For this we will use kallisto. It is a fast and precise allginer.<br>
<br>
Kallisto can be run with just a single command. It requires: <br>
-i a reference genome<br>
-o a place to put the output folder<br>
-b the number of bootstraps kallist will perform (the number of times the allignemnt will be refined)<br>
-- single (or paired if you have paired reads)<br>
-l the avarge length of your reads<br>
-s the standard deviation of your reads<br>
and lastly the file you want to allign.<br>
Optionally you can add "--pseudobam" before giving the file you want to allign. This wil give the output also in the .bam format, which contains the information of where exactly each read is alligned.<br>
<br>
The -l and -s can be extracted from you data set using the following command

In [ ]:
%%bash
cd filtered
for filename in *filtered.fastq; do cat $filename | awk '{if(NR%4==2) print length($1)}' > readlength/$filename.txt; done
#this will count the number of characters in every 4th row (the row with the basepairs) and print them in an output file
ls

In [ ]:
#these files can now be loaded into R. Where you can extract the mean and SD. <br>
#Be sure to first set your working directory. This proccs, I am sure can be looped but I can't for the life of me figure out how. so we will just use the power op "copy" and "paste" 

#Do this in R!
x1 <- read.csv("filtered/readlength/S01subset.fastq-filtered.fastq.txt", header = FALSE)
x2 <- read.csv("filtered/readlength/S02subset.fastq-filtered.fastq.txt", header = FALSE)
x3 <- read.csv("filtered/readlength/S03subset.fastq-filtered.fastq.txt", header = FALSE)
x4 <- read.csv("filtered/readlength/S04subset.fastq-filtered.fastq.txt", header = FALSE)
x5 <- read.csv("filtered/readlength/S05subset.fastq-filtered.fastq.txt", header = FALSE)
x6 <- read.csv("filtered/readlength/S06subset.fastq-filtered.fastq.txt", header = FALSE)
mean(x1$V1); sd(x1$V1)
mean(x2$V1); sd(x2$V1)
mean(x3$V1); sd(x3$V1)
mean(x4$V1); sd(x4$V1)
mean(x5$V1); sd(x5$V1)
mean(x6$V1); sd(x6$V1)

In [ ]:
%%bash
#first we need to make an index. This file can get quite big (1.02gb). 
#A star will appear in the top right corner of this window. when it become a number it is done making the file (5 minutes)
kallisto index -i TAIR10_cdna_20101214_updated.idx TAIR10_cdna_20101214_updated.txt
ls

In [ ]:
%%bash
#in our case the read length and SD is different in every sample, so we will copy paste the command
#running this might take some time, as the library made by kallisto is that of the whole arabidopsis genome everytime.
cd filtered
ls
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S01 -b 5 --single -l 97.143 -s 37.184 S01subset.fastq-filtered.fastq
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S02 -b 5 --single -l 88.706 -s 36.975 S02subset.fastq-filtered.fastq
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S03 -b 5 --single -l 86.015 -s 37.300 S03subset.fastq-filtered.fastq
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S04 -b 5 --single -l 109.709 -s 34.396 S04subset.fastq-filtered.fastq
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S05 -b 5 --single -l 92.368 -s 34.502 S05subset.fastq-filtered.fastq
kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/S06 -b 5 --single -l 94.864 -s 37.371 S06subset.fastq-filtered.fastq


In [ ]:
%%bash
# if all your read lengths and sd are the same (this will happen in the case of illumina) you can loop it.
cd filtered
ls
for filename in S01*filtered.fastq; 
do kallisto quant -i ../TAIR10_cdna_20101214_updated.idx -o ../alligned/$filename -b 5 --single -l 90 -s 30 $filename
echo "done with" $filename
done

Now that we have our allignements we can find how many reads have alligned to every transcript in the "abundance.tsv" files. You can check out the differential expression in your favorite tool, including excel. But we will continue in Sleuth as it is uniquely capable of using the bootstrapping information (abundance.h5) as well.

Optionally if you made the BAM files and what to use these for the <u>UCSC</u> analysis, then you can use the following lines  to get the data ready to be uploaded 

In [ ]:
samtools view -b {input.bam} | bedtools genomecov -split -bg -ibam stdin -g {input.genome} > {output.bg}
bedSort {input.bg} {output.sorted.bg}
bedGraphToBigWig {input.bedgraph} {input.chroms} {output.bw}

# Sleuth

Sleuth is a package that is run in R. Sleuth can compare your sample in 2 ways, using a likelyhood ratio test(lrt), or a wald test(wt). The Wald test will give you a fold change between genes, so that is the test we will use in this case.<br>
We will now go step by step through the procces of doing a wt test. You can copy each piece into R after each other so you can see what everyhting does.

In [1]:
#Do this in R!
#If you haven't already. first install Sleuth. Just copy paste the following command in R and you should be good to go.
source("http://bioconductor.org/biocLite.R")
biocLite("rhdf5")
install.packages("devtools")
devtools::install_github("pachterlab/sleuth")

In [ ]:
#Do this in R!

#load sleuth
suppressMessages({
  library("sleuth")
})

#set work directory yourself first
setwd("~/Documents/RNA-seq/lesson")

#make folders to save to
dir.create(file.path("sleuth_outputWT"))

#Variables. You can make a lot of variables in this run, so you have to change less and less when you want to compare your samples in a different way.
#in this case we will use only 2. The thing we want to compare (here options would be: genotype, condition, tissue, mutant)
variable <- "condition"
#and the thing you want to set as a baseline for your other samples to compare to, often this is WT or control
baseline <- "control"

#First Sleuth needs to know where the files are located.
sample_id <- dir(file.path("alligned"))

#Next we make a file in which the filepaths are located.
kal_dirs <- file.path("alligned", sample_id)

#We give sleuth the experimental set up
s2c <- read.table(file.path("experimental_setup.txt"), header = TRUE, stringsAsFactors=FALSE)
#We tell it over which variable to find the difference. In this case it will “condition”
s2c <- dplyr::select(s2c, sample = sample, variable)
#add filepath for files in pathname so R knows which file corresponds to which metadata entry
s2c <- dplyr::mutate(s2c, path = kal_dirs)

#You can visualize the addition by looking at the table
s2c


#Before we can do the WT test, we need change the s2c file  so that is can easily be read by sleuth.
condfactor <- s2c[,variable]
variablebaseline <- paste (variable,baseline, sep = "", collapse = NULL)
#makes a table for WT test
cond <- factor(condfactor)
cond <- relevel(cond, ref = baseline)
md <- model.matrix(~cond,s2c)
colnames(md)[2] <- variablebaseline


#Next we make the sleuth object. This is where all the information about the comparison is going to be in. 
#you will get a warning, this is because R uses only 1 core of your, probably, multicore computer. When running sleuth in the shell this is not the case. But since runnning from R is easier we will use R anyway.
so <- sleuth_prep(s2c, extra_bootstrap_summary = TRUE)


#sleuth test on differences using the full and reduced model, don't really understand. you have to fill in the "condition" so it knows where to look for differences
so <- sleuth_fit(so, md, 'full')
so <- sleuth_fit(so, ~1, 'reduced')

#And then the walt test is done.
so <- sleuth_wt(so, variablebaseline, 'full')


#If me make a “model” of this sleuth object we can read it.
 models(so)
#make a table of the restults
resultsWT_table <- sleuth_results(so, variablebaseline, test_type = baseline)

#sleuth gives a “b” value. This is a “bias-estimator”, it is close to a fold change when the variance is low (bootstrap and variance between samples of the same group), but if the variance is high it will lower this number, to give an indication that is not very certain of the result. Furthermore the b value is calculated with a natural logarithm instead of LOG2, which is the standard. We will add the log2 b value next
#first take e^x to get rid of the natural logarithm
resultsWT_table$b_e <- exp((resultsWT_table$b))
#Then transform with log2
resultsWT_table$b_e_log2 <- log2(resultsWT_table$b_e)


#write the result to a file
write.table(resultsWT_table, file.path("sleuth_outputWT","WtCvsWtD.txt"), sep="\t") 
#only take the significant part of those files, (q value < 0.05) write them to a new file and show some" 
resultsWT_significant <- dplyr::filter(resultsWT_table, qval <= 0.05)
write.table(resultsWT_significant, file.path("sleuth_outputWT","WtCvsWtDsig.txt"), sep="\t", row.names = FALSE) 

#Lastly we can open up shiny which will allow us to inspect the data in more detail.
sleuth_live(so)

The sleuth object can also be found (Finished_sleuth_analysis.RData)in the lessen plan if you only want to experiment with the shiny visualisation.<br>
under maps, PCA, you can see if the samples map together according to the experimental design.<br>
under analysis, transcript view, you can look up some of the differntielly expressed genes that are saved in the WtCvsWtDSig.txt file. Examples to ook up are: <br>
AT5G46110.1 low under drought <br>
AT1G13930.1 high under drought <br>
AT5G25980.2 (has lots of techinical variance, probably due to 3 spliceforms, which normally is not a lot but since we are working with a low number of reads it is.)